In [ ]:
from rdkit.Chem import rdFingerprintGenerator
from googleapiclient.http import MediaIoBaseDownload
import random
from rdkit import Chem
import pandas as pd
import numpy as np
import zipfile
import io
import sys
import os

sys.path.append("/aloy/home/acomajuncosa/Ersilia/ready-to-screen-enamine-real/src")
from src import download

root = '.'
data_dir = os.path.join(root, "..", "data")
checks_dir = os.path.join(data_dir, "checks")
tmp_dir = os.path.join(root, "..", "tmp")
tmp_dir = '/home/acomajuncosa/Documents/tmp'
FOLDER_ID_ECFP = "1FBELagBf9hlKVgvkaZ8YF60jKRAmsHPo"
PATH_TO_SERVICE = os.path.join(data_dir, "service.json")
os.makedirs(checks_dir, exist_ok=True)

# Manually create a list with all files we must find in 'libraries'
files_in_libraries = []
files_in_libraries += [f"Enamine_REAL_NaturalProducts_{str(i).zfill(3)}.tsv.zip" for i in range(52)]  # 517,797,846 compounds
files_in_libraries += [f"Enamine_REAL_Sample_{str(i).zfill(3)}.tsv.zip" for i in range(104)]  # 1,035,352,518 compounds
files_in_libraries += [f"Enamine_REAL_LeadLike_{str(i).zfill(3)}.tsv.zip" for i in range(838)]  # 8,371,778,942 compounds

random.seed(42)

for FILENAME in files_in_libraries[35:]:

    # Define some variables
    FILENAME = FILENAME.replace(".tsv.zip", "")
    FILE_SMILES_IDS = FILENAME + "_SMILES_IDs.tsv.zip"
    FILE_X = FILENAME + "_X.npz"

    print("\n\n\n-------------------------------------------------------------------------------")
    print(FILENAME)

    # Download files
    download(FILE_SMILES_IDS, os.path.join(tmp_dir, FILE_SMILES_IDS), PATH_TO_SERVICE, FOLDER_ID_ECFP)
    download(FILE_X, os.path.join(tmp_dir, FILE_X), PATH_TO_SERVICE, FOLDER_ID_ECFP)

    # Exctract files from ZIP
    with zipfile.ZipFile(os.path.join(tmp_dir, FILE_SMILES_IDS), "r") as zf:
        zf.extractall(tmp_dir)

    # Load ECFP6s
    N = 10_000
    INDS = random.sample(range(10_000_000), N)
    X = np.load(os.path.join(tmp_dir, FILE_X))['X']  # Caution: >20GB of RAM
    len_X = len(X)
    X = X[INDS]

    # Load SMILES
    SMILES = pd.read_csv(os.path.join(tmp_dir, FILE_SMILES_IDS.replace('.zip', '')), sep='\t')
    len_SMILES = len(SMILES)
    SMILES = np.array(SMILES['smiles'])[INDS]

    print(f"Total number of SMILES: {len_SMILES}")
    print(f"Total number of ECFP6s: {len_X}")

    # Calculate ECFP6s by hand
    MFPS = []
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)
    for smi in SMILES:
        mol = Chem.MolFromSmiles(smi)
        ecfp = mfpgen.GetCountFingerprint(mol)
        arr = np.zeros(2048, dtype=np.int8)
        for i, v in ecfp.GetNonzeroElements().items():
            arr[i] = v
        MFPS.append(arr)
    MFPS = np.array(MFPS)

    # Check if arrays are equal
    ARRAY_EQUAL = np.array_equal(X, MFPS)
    print(f"Are arrays equal? {ARRAY_EQUAL}")

    # Remove files
    os.remove(os.path.join(tmp_dir, FILE_SMILES_IDS))
    os.remove(os.path.join(tmp_dir, FILE_SMILES_IDS.replace(".zip", "")))
    os.remove(os.path.join(tmp_dir, FILE_X))

    # Print results
    with open(os.path.join(checks_dir, f"{FILENAME}_checks.csv"), "w") as out:
        out.write(",".join([FILENAME, str(len_SMILES), str(len_X), str(len(INDS)), str(ARRAY_EQUAL)]))




-------------------------------------------------------------------------------
Enamine_REAL_NaturalProducts_035


Download 8%
Download 17%
Download 26%
Download 35%
Download 44%
Download 53%
Download 62%
Download 71%
Download 80%
Download 88%
Download 97%
Download 100%
Download 1%
Download 2%
Download 3%
Download 4%
Download 6%
Download 7%
Download 8%
Download 9%
Download 10%
Download 12%
Download 13%
Download 14%
Download 15%
Download 16%
Download 18%
Download 19%
Download 20%
Download 21%
Download 22%
Download 24%
Download 25%
Download 26%
Download 27%
Download 29%
Download 30%
Download 31%
Download 32%
Download 33%
Download 35%
Download 36%
Download 37%
Download 38%
Download 39%
Download 41%
Download 42%
Download 43%
Download 44%
Download 45%
Download 47%
Download 48%
Download 49%
Download 50%
Download 51%
Download 53%
Download 54%
Download 55%
Download 56%
Download 58%
Download 59%
Download 60%
Download 61%
Download 62%
Download 64%
Download 65%
Download 66%
Download 67%
Download 68%
Download 70%
Download 71%
Download 72%
Download 73%
Download 74%
Download 76%
Download 77%
Download 78%
Downloa

Total number of SMILES: 10000000
Total number of ECFP6s: 10000000
Are arrays equal? True



-------------------------------------------------------------------------------
Enamine_REAL_NaturalProducts_036


Download 8%
Download 17%
Download 26%
Download 34%
Download 43%
Download 52%
Download 61%
Download 69%
Download 78%
Download 87%
